In [3]:
#Etape 1 : Importer la dataset en créer une dataframe
import pandas as pd
df = pd.read_csv("./Dataset/reseau_en_arbre.csv")

#Création d'un Dataframe ne contenant uniquement les informations dont infra_type = a_remplacer
df = df[df["infra_type"] == "a_remplacer"]
df.head()

,id_batiment,nb_maisons,infra_id,infra_type,longueur
10,E000003,1,P007113,a_remplacer,12.193509
11,E000003,1,P007984,a_remplacer,30.057598
12,E000003,1,P007823,a_remplacer,12.290283
465,E000067,1,P006193,a_remplacer,12.186388
466,E000067,1,P002851,a_remplacer,10.803202
